In [ ]:
from IPython.display import display
import ipywidgets as widgets
from simpleac import SimPleAC
from monte_carlo import monte_carlo_results
from datetime import datetime as dt
import plotly.graph_objects as go

button = widgets.Button(description="Run Simulation")

#--CONDITION SPECIFIC--START
wing_weight_margin = widgets.FloatSlider(description = "Wing Weight Margin",
                                  value = 1.125)
tsfc_margin = widgets.FloatSlider(description = "TSFC Margin", #Thrust-Specific Fuel Consumption
                                value = 1)
takeoff_speed_margin = widgets.FloatSlider(description = "Takeoff Speed Margin",
                                            value = 1)
range_margin = widgets.FloatSlider(description = "Range Margin",
                                   value = 1)
levers = [wing_weight_margin, tsfc_margin, takeoff_speed_margin, range_margin]
#--CONDITION SPECIFIC--END

for lever in levers:
    lever.style = {'description_width': '200px'}
    lever.layout.width = '800px'
    lever.continuous_update = True

progress = widgets.FloatProgress(value=0.0,
                                 min=0.0,
                                 max=1.0, 
                                 description='Loading:',
                                 bar_style='info',
                                 orientation='horizontal')
progress.layout.visibility = 'hidden'

out = widgets.Output(layout={'border': '1px solid black'})

fig = go.FigureWidget()
fig.add_scatter();

In [ ]:
x = []
y = []
conds = []
times = []
start_time = dt.now()

def on_button_clicked(b):
    m = SimPleAC()
    #--CONDITION SPECIFIC--START
    m.substitutions.update({
        "m_ww": wing_weight_margin.value,
        "m_tsfc": tsfc_margin.value,
        "m_vmin": takeoff_speed_margin.value,
        "m_range": range_margin.value
    })
    #--CONDITION SPECIFIC--END
    progress.value = 0
    progress.layout.visibility = None
    out.clear_output()
    with out:
        print("Conditions: " + str([lever.value for lever in levers]))
    performance, failure = monte_carlo_results(m, progress, out)
    if performance:
        x.append(performance)
        y.append(failure)
        conds.append("Conditions: " + str([lever.value for lever in levers]))
        fig.data[0].x = x
        fig.data[0].y = y
        fig.data[0].hovertext = conds
        times.append(dt.now())
    progress.layout.visibility = 'hidden'

button.on_click(on_button_clicked)

In [ ]:
widgets.VBox(levers+[button, fig, progress, out])